In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from talib import BBANDS, RSI, MA_Type, ATR
import datetime as dt

In [22]:
# # Download historical data
# # Define the start and end dates for the data
# end_date = dt.datetime.now()
# start_date = end_date - dt.timedelta(days=729)

# # Download 5-minute interval data for the specified date range
# df = yf.download('TSLA', start=start_date, end=end_date, interval='1h')

# print(df.head())

tickerSymbol = 'TSLA'
# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)
# Get the historical prices for this ticker
df = tickerData.history(period='60d', interval='5m')
# Print the data
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-03-07 09:30:00-05:00,191.380005,192.243896,190.320007,191.809998,6399407,0.0,0.0
2023-03-07 09:35:00-05:00,191.821503,192.389999,191.500000,191.787903,2630290,0.0,0.0
2023-03-07 09:40:00-05:00,191.820007,192.203003,190.850006,191.029999,2650631,0.0,0.0
2023-03-07 09:45:00-05:00,191.000000,191.752396,191.000000,191.229996,1874939,0.0,0.0
2023-03-07 09:50:00-05:00,191.279999,191.339996,190.610001,190.986893,2051243,0.0,0.0


In [3]:
# Calculate Bollinger Bands
df['upper_band'], df['middle_band'], df['lower_band'] = BBANDS(df['Close'], timeperiod=20)

# Calculate RSI
df['rsi'] = RSI(df['Close'], timeperiod=14)

# Calculate Keltner Channels
df['keltner_middle'] = df['Close'].rolling(window=20).mean()
df['atr'] = ATR(df['High'], df['Low'], df['Close'], timeperiod=20)
df['keltner_upper'] = df['keltner_middle'] + (df['atr'] * 1.5)
df['keltner_lower'] = df['keltner_middle'] - (df['atr'] * 1.5)


In [4]:
# Set starting balance and risk amount
starting_balance = 1000000
risk_amount = (.05 * starting_balance) # 5% of starting balance
stop_loss_pct = 0.10 # 10%
balance = starting_balance


In [5]:
# Initialize variables
position = 0
entry_price = 0
num_shares = 0

In [6]:
# Initialize trade journal
trade_journal = pd.DataFrame(columns=["Date", "Action", "Price", "Shares", "Balance"])


In [14]:
# Iterate over the data
for i, row in df.iterrows():
    # Check if RSI is below 20 and close price is below the lower Keltner Channel and lower Bollinger Band
    if row['rsi'] < 20 and (row['Close'] <  row['keltner_lower'] or row['Close'] < row['lower_band']):
        if position == 0:
            # Calculate the number of shares we can buy based on risk amount and current price
            num_shares = int(risk_amount / row['Close'])
            # Subtract the cost from our balance
            balance -= num_shares * row['Close']
            # Update our position
            position += num_shares
            # Update the entry price
            entry_price = row['Close']
            trade_journal = trade_journal.append({"Date": row.name, "Action": "Buy", "Price": row["Close"], "Shares": num_shares, "Balance": balance}, ignore_index=True)

    # Check if RSI is above 80 and close price is above the upper Keltner Channel and upper Bollinger Band, or the stop loss is triggered
    elif (row['rsi'] > 75 and (row['Close'] > row['keltner_upper'] or row['Close'] > row['upper_band'])) or (entry_price > 0 and row['Close'] < entry_price * (1 - stop_loss_pct)):
        if position > 0:
            # Add the revenue from selling our position to our balance
            balance += position * row['Close']
            trade_journal = trade_journal.append({"Date": row.name, "Action": "Sell", "Price": row["Close"], "Shares": position, "Balance": balance}, ignore_index=True)
            # Update our position
            position = 0

In [16]:
print(f'Final balance: {balance}')

Final balance: 1019448.7903137207


In [17]:
# Separate dataframes for buy and sell operations
buy_df = trade_journal[trade_journal["Action"] == "Buy"].copy().reset_index(drop=True)
sell_df = trade_journal[trade_journal["Action"] == "Sell"].copy().reset_index(drop=True)

In [18]:
# Ensure that buy_df and sell_df have the same length
if len(buy_df) != len(sell_df):
    min_len = min(len(buy_df), len(sell_df))
    buy_df = buy_df[:min_len]
    sell_df = sell_df[:min_len]

In [19]:
# Calculate metrics
num_wins = (sell_df["Price"] > buy_df["Price"]).sum()
num_losses = (sell_df["Price"] < buy_df["Price"]).sum()
win_ratio = num_wins / (num_wins + num_losses)
total_profit = balance - starting_balance
num_trades = len(buy_df)

print(f'Number of trades: {num_trades}')
print(f'Win/Loss ratio: {win_ratio}')
print(f'Total profit/loss: {total_profit}')
print(trade_journal)

Number of trades: 24
Win/Loss ratio: 0.9166666666666666
Total profit/loss: 19448.790313720703
                        Date Action       Price Shares       Balance
0  2023-03-08 09:30:00-05:00    Buy  183.630005    272  9.500526e+05
1  2023-03-09 09:45:00-05:00   Sell  184.639999    272  1.000275e+06
2  2023-03-10 09:40:00-05:00    Buy  170.149994    293  9.504208e+05
3  2023-03-13 14:30:00-04:00   Sell  177.234299    293  1.002350e+06
4  2023-03-23 14:40:00-04:00    Buy  188.750000    264  9.525204e+05
5  2023-03-27 09:30:00-04:00   Sell  196.251999    264  1.004331e+06
6  2023-03-28 09:35:00-04:00    Buy  188.750305    264  9.545009e+05
7  2023-03-29 09:30:00-04:00   Sell  194.132706    264  1.005752e+06
8  2023-04-03 09:50:00-04:00    Buy  198.089996    252  9.558332e+05
9  2023-04-04 09:30:00-04:00   Sell  198.440002    252  1.005840e+06
10 2023-04-06 09:30:00-04:00    Buy  180.460007    277  9.558527e+05
11 2023-04-11 09:30:00-04:00   Sell  187.860107    277  1.007890e+06
12 2023-0